In [1]:
import sys
sys.path.append("..")

from scripts.utils import DuckdbUtils

duckdb_utils = DuckdbUtils()

In [2]:
sql_query = """
SHOW tables
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,gold_2023_agency_fee_metrics
4,gold_2023_county_violations_day_of_week_heatmap
5,gold_2023_precinct_violations_day_of_week_heatmap
6,gold_2023_ticket_counts_year_month
7,gold_2023_ticket_counts_year_week
8,gold_2023_violations_day_of_week_heatmap
9,gold_2023_weekly_violations_day_of_week_heatmap


In [2]:
sql_query = """
WITH table_metrics AS (
    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        SUM(fee_usd) AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '4.silver_valid_violation_tickets' AS table_name
    FROM
        silver_valid_violation_tickets

    UNION

    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        SUM(fee_usd) AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '3. silver_violation_tickets' AS table_name
    FROM
        silver_violation_tickets

    UNION

    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        'Value not available' AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '2. silver_parking_violations' AS table_name
    FROM
        silver_parking_violations

    UNION

    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        'Value not available' AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '1. bronze_parking_violations' AS table_name
    FROM
        bronze_parking_violations
)
SELECT * FROM table_metrics ORDER BY table_name ASC
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,unique_precinct_count,unique_violation_code_count,unique_violation_county_count,unique_issuing_agency_count,total_revenue,earliest_issue_date,latest_issue_date,table_name
0,80,76,17,25,Value not available,2000-05-21,2029-09-11,1. bronze_parking_violations
1,80,76,17,25,Value not available,2000-05-21,2029-09-11,2. silver_parking_violations
2,80,76,17,25,7273820,2000-05-21,2029-09-11,3. silver_violation_tickets
3,77,66,13,21,2344815,2023-01-01,2023-08-30,4.silver_valid_violation_tickets


In [3]:
sql_query = """
WITH table_metrics AS (
    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        SUM(fee_usd) AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '4.silver_valid_violation_tickets' AS table_name
    FROM
        silver_valid_violation_tickets
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8

    UNION

    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        SUM(fee_usd) AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '3. silver_violation_tickets' AS table_name
    FROM
        silver_violation_tickets
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8

    UNION

    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        'Value not available' AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '2. silver_parking_violations' AS table_name
    FROM
        silver_parking_violations
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8

    UNION

    SELECT 
        COUNT(DISTINCT violation_precinct) AS unique_precinct_count,
        COUNT(DISTINCT violation_code) AS unique_violation_code_count,
        COUNT(DISTINCT violation_county) AS unique_violation_county_count,
        COUNT(DISTINCT issuing_agency) AS unique_issuing_agency_count,
        'Value not available' AS total_revenue,
        MIN(issue_date) AS earliest_issue_date,
        MAX(issue_date) AS latest_issue_date,
        '1. bronze_parking_violations' AS table_name
    FROM
        bronze_parking_violations
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8
)
SELECT * FROM table_metrics ORDER BY table_name ASC
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,unique_precinct_count,unique_violation_code_count,unique_violation_county_count,unique_issuing_agency_count,total_revenue,earliest_issue_date,latest_issue_date,table_name
0,78,70,17,22,Value not available,2023-01-01,2023-08-30,1. bronze_parking_violations
1,78,70,17,22,Value not available,2023-01-01,2023-08-30,2. silver_parking_violations
2,78,70,17,22,3997470,2023-01-01,2023-08-30,3. silver_violation_tickets
3,77,66,13,21,2344815,2023-01-01,2023-08-30,4.silver_valid_violation_tickets


In [4]:
sql_query = """
WITH silver_violation_tickets_dq_check AS (
    SELECT DISTINCT
        violation_precinct,
        '3.silver_violation_tickets' AS table_name
    FROM
        silver_violation_tickets
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8
),

silver_valid_violation_tickets_dq_check AS (
    SELECT DISTINCT
        violation_precinct,
        '4.silver_valid_violation_tickets' AS table_name
    FROM
        silver_valid_violation_tickets
    WHERE
        EXTRACT(year FROM issue_date) == 2023 AND
        EXTRACT(month FROM issue_date) <= 8
)

SELECT
    silver_violation_tickets_dq_check.violation_precinct,
    silver_violation_tickets_dq_check.table_name AS silver_violation_tickets,
    silver_valid_violation_tickets_dq_check.table_name AS silver_valid_violation_tickets
FROM
    silver_violation_tickets_dq_check
LEFT JOIN
    silver_valid_violation_tickets_dq_check ON
        silver_violation_tickets_dq_check.violation_precinct = silver_valid_violation_tickets_dq_check.violation_precinct
WHERE
    silver_valid_violation_tickets_dq_check.table_name IS NULL
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,violation_precinct,silver_violation_tickets,silver_valid_violation_tickets
0,0,3.silver_violation_tickets,None


In [ ]:
sql_query = """
SHOW tables
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)